In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import pickle

In [ ]:
import autoreload
import evotsc
import evotsc_plot
autoreload.reload(evotsc)
autoreload.reload(evotsc_plot)

In [ ]:
# Constants
intergene = 1000
interaction_dist = 2500
interaction_coef = 0.3
default_basal_expression = 1.0
nb_eval_steps=51
beta_A = 0.15
beta_B = 0.30
nb_genes = 60
nb_indivs = 100
inversion_prob=0.75

In [ ]:
init_genes = evotsc.Gene.generate(intergene=intergene,
                                  nb_genes=nb_genes,
                                  default_basal_expression=default_basal_expression)

In [ ]:
init_indiv = evotsc.Individual(genes=init_genes,
                               interaction_dist=interaction_dist,
                               interaction_coef=interaction_coef,
                               nb_eval_steps=nb_eval_steps,
                               beta_A=beta_A,
                               beta_B=beta_B)

In [ ]:
mutation = evotsc.Mutation(intergene_mutation_prob=0.0,
                           intergene_mutation_var=0.0,
                           inversion_prob=inversion_prob)

In [ ]:
evotsc_plot.plot_expr_AB(indiv=init_indiv, plot_title='best generation 0', plot_name=f'best generation {0:05}')
evotsc_plot.explain(init_indiv)

In [ ]:
population = evotsc.Population(init_indiv=init_indiv, nb_indivs=nb_indivs, mutation=mutation)

In [ ]:
step = 50

In [ ]:
for gen in range(1001):
    best_indiv, avg_fit = population.step()
    
    if gen % 10 == 0:
        print(f'Generation {gen:05}: best fit {best_indiv.fitness:.5}, avg fit {avg_fit:.5}')
    
    if gen % step == 0:
        evotsc_plot.plot_expr_AB(best_indiv, f'best generation {gen:05}', f'best generation {gen:05}.png')
        evotsc_plot.explain(best_indiv)
        print()

In [ ]:
last, avg = population.step()

In [ ]:
evotsc_plot.explain(last)